# Test if SIMIO is working correctly

**This test uses data from** [tutorial 1](https://simio-continuum.readthedocs.io/index.html#document-tutorials/tutorial_1) **, which you can download from** [here](https://keeper.mpdl.mpg.de/d/7fe6524c7ec945168630/)

If you customize your SIMIO functions and you want to test if your modifications have affected the code, there are two major properties you should check: **Flux conservation** from the input model and **size scaling** as a function of distance. Let us use the Solar System from [Bergez-Casalou et al. 2022](https://ui.adsabs.harvard.edu/abs/2022A%26A...659A...6B/abstract) as the testing case for SIMIO functionalities. 

<img src="./SolarS_original.png">
<center> Figure 1: The 1.3mm continuum emission from the young Solar System as simulated by Bergez-Casalou et al. 2022. </center>

## Flux conservation

SIMIO takes the input image and translates it to a "*.model*" image of CASA, before calculating the Fourier Transform with *ft*. During this translation the image is copied and flux-scaled following the inverse square law.

Let us test if this translation worked correctly. The original young Solar System model has a flux of **427.591784Jy** when positioned at 1pc, which is the default distance from RADMC3D outputs and is the assumed distance for the input models of SIMIO (when the flux is not overriden, see [tutorial 5](https://simio-continuum.readthedocs.io/index.html#document-tutorials/tutorial_5)). Let us generate the synthetic observation at the distance of HD163296, which is 100.966pc from [GAIA DR3](https://gea.esac.esa.int/archive/). At such distance, we expect a flux of:

$$
  \frac{427.591784 \,\text{Jy}}{100.966^2} = 0.041944 \,\text{Jy} = 41.944 \,\text{mJy}
$$

```python
# Import needed python packages
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Get the current directory path
current_dir = os.getcwd()+'/'

####################################

# Import the analysis utils functions
sys.path.append(current_dir+'codes/analysis_scripts/')
import analysisUtils as au

# Import the simio object
execfile(current_dir+'codes/simio_obj.py')
# Import functions for uv-handling
execfile(current_dir+'codes/simio_ms2ascii.py')
# Import functions for imaging
execfile(current_dir+'codes/simio_clean.py')

###########################
# Solar System as HD163296
###########################

# Create a simio object.
simobj = simio_object(object_name  = 'SolarS_HD163296', 
                      im_file_name = 'image_1300micron.out', 
                      template     = 'HD163296', 
                      use_tempgeom = True)

# Create the measurement file of your simio object
mod_ms = get_mod_ms_ft(simobj)
```

To generate *mod_ms*, *SIMIO* generated an image called "***SolarS_HD163296_orig_model_im.fits***". The visibilities are calculated from this image. Let us use *python* to read the image and calculate the flux.

```python
# Import astropy.io.fits to read the image in python
from astropy.io import fits

# Path to the image
model_path = 'projects/SolarS_HD163296/images/SolarS_HD163296_orig_model_im.fits'
# Open image
model_im  = fits.open(model_path)[0].data

# Calculate flux
print (np.sum(model_im))

# 0.04194438763274089
```

This value matches the expected value from the inverse square law. As the Fourier Transform is calculated from this image, we confirm that distance correction conserved the flux. Always check the ***project_orig_model_im.fits*** for this test.

## Distance scaling

The same image from the previous part will be used to check distance scaling. The original young Solar System image has 800 pixels and 0.4au of pixel size, therefore the whole image is 320au in length. Let us check the translated image:

```python
# Import astropy.io.fits to read the image in python
from astropy.io import fits

# Distance to HD163296
parallax = 9.90426503087695 # GAIA EDR3 mas
dist     = 1000. / parallax # pc
# Size of the input image
imsize_input = 320.

# Path to the image
model_path = 'projects/SolarS_HD163296/images/SolarS_HD163296_orig_model_im.fits'
# Read header
header  = fits.open(dir_cont)[0].header
# Calculate image size in arcsec
ra_ext  = 3600. * header_cont['CDELT1'] * header_cont['NAXIS1']
dec_ext = 3600. * header_cont['CDELT2'] * header_cont['NAXIS2']

# Calculate image size in au
imsize_au = dec_ext * dist
print (dec_ext * dist)
# 320.0045547174389

# Compare to real image size
ratio_im = dec_ext * dist / imsize_input # 320 is the size in au of the input image
print (ratio_im)
# 1.0000142334919966
```

The difference between the input image size and the model image is of the order of $10^{-5}$ in ratio, and it probably comes from the numerical systematics of transforming the pixel size in radians contained in ```header_cont['CDELT1']``` to arcsec units and then to astronomical units. This way, you can test the distance scaling of your input model.